In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json


from selenium import webdriver
from selenium.webdriver.common.by import By


In [11]:
# 데이터를 저장하는 리스트 변수
hospital_data = []

base_url = "https://www.akd.or.kr/search/sub02.php?mode=&mod=Y&keyfield=mb_addr1&key=%EC%84%9C%EC%9A%B8&key2=&key3=&key4=&sname=&sno=&stel=&member_level=&list_per_page=10&"
max_limit = 910
chrome = webdriver.Chrome()
time.sleep(5)

for idx, start in enumerate(range(0, max_limit + 1, 10)):
    full_url = f"{base_url}start={start}#11"
    response = requests.get(full_url)

    if response.status_code != 200:
        print(f"실패 start={start}")
        break
    
    chrome.get(full_url)
    time.sleep(0.5)

    soup = BeautifulSoup(chrome.page_source, "html.parser")

    rows = soup.select("tbody tr")

    for row in rows:
        columns = row.find_all("td")
        if len(columns) < 5:
            continue

        img_tag = columns[1].find("img")
        dermatologist = "Yes" if img_tag else "No"

        hospital_name = columns[2].get_text(strip=True)
        doctor_name = columns[3].get_text(strip=True)
        address = columns[4].get_text(strip=True)
        number = columns[5].get_text(strip=True)

        # { 제품명 : 성분리스트 }를 리스트 변수에 저장
        hospital_data.append({
            "dermatologist": dermatologist,
            "hospital_name" : hospital_name,            
            "doctor_name" : doctor_name,
            "address" : address,
            "number": number
        })

with open('hospital_data.json', 'w', encoding='utf-8') as f:
    json.dump(hospital_data, f, ensure_ascii=False, indent=4)

In [12]:
chrome.close()
chrome.quit()

In [13]:
with open('hospital_data.json', 'r', encoding='utf-8') as f:
    hospital_data = json.load(f)

print(len(hospital_data))

915
